In [11]:
# Requirements
import json
import time
import csv
import requests
import os
import yaml
from utils.checkTweetsForDuplicates import check_duplicate_ids
from pprint import pprint
from configparser import ConfigParser
from random import randint

In [4]:
# Debug mode for testing
DEBUG_MODE = True

In [5]:
# Reading api_key from config.ini
api_key: str | None = None
try:
    print("Reading api-key from config.ini ...")
    config = ConfigParser()
    config.read('../conf/config.ini') # adjust the path to config.ini
    api_key = config['TWITTERAPI']['API_KEY']
    print(f"API Key gefunden: {api_key[:5]} ...")

except Exception as e:
    print("Failed to read credentials. Exit with error:")
    print(e)

Reading api-key from config.ini ...
API Key gefunden: 1228a ...


In [6]:
# Declare vars
if api_key is None:
    raise ValueError("Could not read api key.")

# Array to store tweets
all_tweets = []

# Parameters for api request
headers = {"X-API-Key": api_key}
url = "https://api.twitterapi.io/twitter/tweet/advanced_search"
cursor = ""

# counter
i = 1

In [17]:
# Creating cursor file

# Storing latest cursor position
actual_cursor = "DAADDAABCgABGp0h4-_aYKIKAAIaN7kUXtZB8QAIAAIAAAACCAADAAAAAAgABAAAARsKAAUaoHI5EIAnEAoABhqgcjkQVKpAAAA"
cursor_location = "../utils/cursor.yml"

if not os.path.exists("../utils"):
    print("No previous cursor stored. Making file and storing last location ...")
    os.mkdir("../utils")

# If existing, read file content
try:
    with open(cursor_location, 'r') as file:
        cursor_data = yaml.safe_load(file) or {}
except FileNotFoundError:
    cursor_data = {}

# Construct new cursor data
page_key = "page_284"
page_data = {
    "tweets_per_page": 20,
    "total_tweet_count": 5668,
    "next_page": actual_cursor
}

if page_key in cursor_data:
    print(f"Page '{page_key}' already in file. Next page @ ...{actual_cursor[-10:]}")
else:
    # Add entry to (existing) data
    cursor_data[page_key] = page_data
    print("Adding following cursor data to file:")
    pprint(cursor_data)

    with open(cursor_location, "w", encoding='utf-8') as f:
        yaml.dump(cursor_data, f, allow_unicode=True, sort_keys=True)
        print("Adding data successful. Continuing ...")

Page 'page_284' already in file. Next page @ ...jkQVKpAAAA


In [7]:
# https request
# Endless loop based on has_next_page - break when false
while True:
    try:
        # Make https request
        response = requests.request(
            "GET",
            url,
            headers=headers,
            params={
                "query": "from:elonmusk since:2023-05-01 until:2025-05-01 -is:retweet",
                "cursor": cursor
                # querytype: "latest" by default
            }
        )

        if response.status_code == 200:
            # Debugging infos
            print(f"Status Code: {response.status_code}")

            # throw exception for http error
            response.raise_for_status()

            # Store response
            json_data = response.json()

            # Extract necessary vars
            tweets = json_data.get("tweets", [])
            tweet_ids = [tweet["id"] for tweet in tweets]
            has_next_page = json_data.get("has_next_page")
            next_cursor = json_data.get("next_cursor")

            # Store and debug tweets
            prev_tweet_count = len(all_tweets)
            all_tweets.extend(tweets)
            new_tweet_count = len(all_tweets) - prev_tweet_count
            print(f"New Tweets in iteration {i}: {new_tweet_count}")
            print(f"Total Tweet count: {len(all_tweets)}")

            # Break if debug mode is active
            if DEBUG_MODE:
                debug_mode_tweets = json_data["tweets"][new_tweet_count - 1]
                print(f'DEBUG_MODE active: stopping after first page. Got {new_tweet_count} tweets:')
                pprint(debug_mode_tweets)
                break

            if not has_next_page:
                print(f"Got {new_tweet_count} tweets. Finished.")
                break

            # Prepare the next iteration
            print(f"Next page detected @ '{next_cursor[:10]}'. Updating cursor ...")
            cursor = next_cursor
            i += 1

            # Wait random time to avoid rate limits
            delay = randint(1, 5)
            print(f"Starting new iteration in {delay} seconds ...")
            time.sleep(delay)

    except Exception as e:
        print("Error when requesting and processing tweets:")
        print(e)
        break



Status Code: 200
New Tweets in iteration 1: 20
Total Tweet count: 20
Next page detected @ 'DAADDAABCg'. Updating cursor ...
Starting new iteration in 4 seconds ...
Status Code: 200
New Tweets in iteration 2: 20
Total Tweet count: 40
Next page detected @ 'DAADDAABCg'. Updating cursor ...
Starting new iteration in 2 seconds ...
Status Code: 200
New Tweets in iteration 3: 20
Total Tweet count: 60
Next page detected @ 'DAADDAABCg'. Updating cursor ...
Starting new iteration in 4 seconds ...
Status Code: 200
New Tweets in iteration 4: 20
Total Tweet count: 80
Next page detected @ 'DAADDAABCg'. Updating cursor ...
Starting new iteration in 2 seconds ...
Status Code: 200
New Tweets in iteration 5: 20
Total Tweet count: 100
Next page detected @ 'DAADDAABCg'. Updating cursor ...
Starting new iteration in 2 seconds ...
Status Code: 200
New Tweets in iteration 6: 20
Total Tweet count: 120
Next page detected @ 'DAADDAABCg'. Updating cursor ...
Starting new iteration in 1 seconds ...
Status Code: 2

KeyboardInterrupt: 

In [2]:
# Check for duplicates in tweets
data_location = "../data/twitter/tweets.csv"

if not os.path.exists(data_location):
    print("File does not exist. Exiting ...")
    exit()

try:
    has_duplicates = check_duplicate_ids(path)
    print(f"Has duplicates: {has_duplicates}")
except Exception as e:
    print("Could not call function. Exit with error:")
    print(e)

print("\nNext cursor after manually stopping scraping:")
print(cursor)

No duplicates found.
Has duplicates: False

Next cursor after manually stopping scraping:


NameError: name 'cursor' is not defined

In [9]:
# Writing tweets to file
try:
    print('Finished request. Converting results to file ...')

    # Create the dir if it does not exist
    if not os.path.exists("../data/twitter"):
        os.mkdir("../data/twitter")

    # Target location
    filepath = "../data/twitter/tweets.csv"

    # Store existing tweet ids as set and write all existing ones
    existing_ids = set()
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8", newline='') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_ids.add(row["id"])

    # Open the file in append mode
    with open(filepath, "a", encoding="utf-8", newline='') as f:
        if not all_tweets:
            print("No tweets to write. Exiting ...")
            exit()

        # Flatten tweet structure
        def flatten_tweet(tweet):
            if not tweet:
                return {}

            flat = {}
            for k, v in tweet.items():
                try:
                    if isinstance(v, (dict, list)):
                        flat[k] = json.dumps(v, ensure_ascii=False)
                    else:
                        flat[k] = v
                except Exception as e:
                    print(f"Error when processing field {k}: {e}")
                    flat[k] = None
            return flat

        flattened_tweets = [flatten_tweet(tw) for tw in all_tweets]
        if not flattened_tweets:
            print("Did not find tweets to write. Exiting ...")
            exit()

        # Derive col names from the first tweet
        fieldnames = list(flattened_tweets[0].keys())

        # Create a csv writer with fieldnames
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if os.stat(filepath).st_size == 0:
            writer.writeheader()

        # Write new tweets to a file
        new_count = 0
        for tweet in flattened_tweets:
            if tweet["id"] not in existing_ids:
                writer.writerow(tweet)
                new_count += 1

        print(f"Wrote {new_count} new tweets to CSV.")

except Exception as e:
    print('Failed to convert tweets to file. Exit with error:')
    print(e)

Finished request. Converting results to file ...
Wrote 5648 new tweets to CSV.


In [150]:
# Working directory
# If the previous cell says it cannot find the location, run this first
# os.chdir(os.path.expanduser("~/PycharmProjects/COIN_Repo")) # adjust path to root dir of project
# os.getcwd() # Should be root dir now